# Elementy Inteligencji Obliczeniowej - Sieci Neuronowe


---

**Prowadzący:** Jakub Bednarek<br>
**Kontakt:** jakub.bednarek@put.poznan.pl<br>
**Materiały:** [Strona WWW](http://jakub.bednarek.pracownik.put.poznan.pl)

---

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Uwaga

* **Aby wykonać polecenia należy najpierw przejść do trybu 'playground'. File -> Open in Playground Mode**
* Nowe funkcje Colab pozwalają na autouzupełnianie oraz czytanie dokumentacji

## Technologie:

Na zajęciach będziemy korzystać z języka Python oraz bibliotek **NumPy**, **Tensorflow**, IPython, Pillow, Matplotlib. Ponieważ:
* obliczenia wykonywane przez sieci są masywne - przez co rozsądne jest aby korzystać z GPU,
* nie każdy ma GPU w swoim komputerze,
* nie każdy chce zostawać po godzinach w laboratoriach i korzystać z uczelnianych GPU,
* konfiguracja Tensorflow na GPU potrafi być uciążliwa,

będziemy korzystać głównie z Colab, na którym jest skonfigurowane środowisko wraz ze wszystkimi potrzebnymi składnikami oraz GPU.

*Dla zainteresowanych: na stronie [Tensorflow](https://www.tensorflow.org/install/) znajduje się poradnik jak zainstalować bibliotekę na własnym komputerze. Popularnym podejściem do konfiguracji środowiska TensorFlow na własnej maszynie jest instalacja wirtualnego środowiska przy wykorzystaniu oprogramowania **Anaconda (Conda)**. Popularne IDE do Pythona, jak PyCharm posiadają obsługę środowisk stworzonych przez Condę. Można również korzystać z [NVIDIA-Docker2](https://github.com/NVIDIA/nvidia-docker) oraz gotowych obrazów systemu dostępnych po rejestracji na [NGC](https://ngc.nvidia.com).*


## Cel ćwiczeń:

* zapoznanie się z językiem Python oraz popularnymi bibliotekami do przetwarzania danych,
* przedstawienie podstaw teoretycznych, na których oparte są sieci neuronowe,
* implementacja podstawowych elementów sieci neuronowej (perceptron, funkcja aktywacji, funkcja straty, warstwa neuronów) i sposobu ich uczenia (**wsteczna propagacja błędu**).

## NumPy

NumPy jest biblioteką przeznaczoną do obliczeń o charakterze naukowym. Zawiera implementacje operacji na macierzach i wektorach, jak i bardziej zaawansowane funkcje typu transformata Fouriera.

Aby korzystać z NumPy należy wykonać następującą instrukcję:

In [3]:
# import biblioteki NumPy i przypisanie aliasu ''np''
import numpy as np

In [4]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


Podstawowym typem danych w NumPy jest tablica, poniżej zostały zaprezentowane różne metody alokacji takich tablic.

In [5]:
# funkcja np.array() zmienia parametr (lista/tuple) na tablicę NumPy,
# jednym z parametrów jest dtype który określa typ alokowanych danych (np.int32, np.float32, itp.)

arr = np.array([1, 2, 3], dtype=np.int32)

print('Tablica int:', arr, '\n')

arr = np.array([1, 2, 3], dtype=np.float32)
print('Tablica float:', arr, '\n')

arr = np.zeros([2, 3], dtype=np.float32)
print('Tablica zer:\n', arr, '\n')

arr = np.ones([2, 3], dtype=np.float32)
print('Tablica jedynek:\n', arr, '\n')

arr = np.random.normal(5, 2, [5])
print('5 wartości z rozkładu normalnego N(5, 2):\n', arr, '\n')

arr = np.random.uniform(0, 2, [5])
print('5 wartości losowych z przedziału <0, 2):\n', arr, '\n')

arr = np.zeros_like(arr, dtype=np.float64)
print('Tablica zer (float64) o takim samym rozmiarze co poprzednia tablica:\n', arr, '\n')

arr = np.zeros_like(arr, dtype=np.uint8)
print('Tablica zer (uint8) o takim samym rozmiarze co poprzednia tablica:\n', arr, '\n')

Tablica int: [1 2 3] 

Tablica float: [1. 2. 3.] 

Tablica zer:
 [[0. 0. 0.]
 [0. 0. 0.]] 

Tablica jedynek:
 [[1. 1. 1.]
 [1. 1. 1.]] 

5 wartości z rozkładu normalnego N(5, 2):
 [3.22034897 3.35805905 4.43777175 2.47506    5.45575877] 

5 wartości losowych z przedziału <0, 2):
 [0.44243379 0.25948167 1.20288341 0.72959183 1.44655531] 

Tablica zer (float64) o takim samym rozmiarze co poprzednia tablica:
 [0. 0. 0. 0. 0.] 

Tablica zer (uint8) o takim samym rozmiarze co poprzednia tablica:
 [0 0 0 0 0] 



Dodatkowo w bibliotece można znaleźć metody alokacji według pewnego wzorca jak *range* czy *linspace* (UWAGA: sprawdź, czy metody zwracają również przypadki krańcowe jako elementy tablicy):

In [6]:
# alokacja tablicy 10 wartości od 0 do 10 z równym krokiem
arr = np.linspace(0, 10, 10)

print('Linspace:', arr, '\n')

# alokacja tablicy z wartościami od 0 do 10 z krokiem 0.5
arr = np.arange(0, 10, 0.5)

print('ARange:', arr, '\n')

Linspace: [ 0.          1.11111111  2.22222222  3.33333333  4.44444444  5.55555556
  6.66666667  7.77777778  8.88888889 10.        ] 

ARange: [0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5 5.  5.5 6.  6.5 7.  7.5 8.  8.5
 9.  9.5] 



Kolejną przydatną funkcją jest funkcja **reshape** do zmiany wymiarów tablicy.

In [7]:
# alokacja danych (tablica 25 wartości)
arr = np.linspace(0, 10, 25)


# zmiana rozmiaru z (25) na (5, 5):
arr = np.reshape(arr, [5, 5])

print(arr)


[[ 0.          0.41666667  0.83333333  1.25        1.66666667]
 [ 2.08333333  2.5         2.91666667  3.33333333  3.75      ]
 [ 4.16666667  4.58333333  5.          5.41666667  5.83333333]
 [ 6.25        6.66666667  7.08333333  7.5         7.91666667]
 [ 8.33333333  8.75        9.16666667  9.58333333 10.        ]]


NumPy dostarcza implementacji wielu przydatnych operacji na tablicach. Poniżej zaprezentowane zostały podstawowe operacje:

In [8]:
# alokacja danych
a = np.random.normal(0, 1, [3, 3])
b = np.random.uniform(-1, 1, [3, 3])

print('Tablica a:\n', a, '\n\nTablica b:\n', b, '\n')

# mnożenie element-wise:
print('Mnożenie elementwise:\n', np.multiply(a, b), '\n')

# mnożenie macierzy:
print('Mnożenie macierzy:\n', np.matmul(a, b), '\n')

# transpozycja tablicy
print('Transpozycja:\n', np.transpose(a, [1, 0]), '\n')

# potęgowanie:
print('Potęgowanie:\n', np.power(a, 2), '\n')

# broadcasted ops (są to operacje które wykonują pewien wzorzec dla wszystkich elementów):
# np. dla macierzy o wymiarach (5, 10) możemy zdefiniować tablicę o wymiarach (1, 10)
# wówczas podstawowe operacje spowodują, że dla każdego wiersza (5 wierszy) zostanie wykonana
# taka operacja z wykorzystaniem 10 elementów drugiej tablicy
# poniżej a ma rozmiar (3, 3), factor ma rozmiar (1, 3)
factor = np.array([[1, 2, 3]], dtype=np.float32)
print('Broadcasted multiply:\n', np.multiply(a, factor), '\n')

# Sprawdź również broadcasted ops dla innych operacji jak dodawanie, odejmowanie.
print('Broadcasted add:\n', np.add(a, factor), '\n')
print('Broadcasted subtract:\n', np.subtract(a, factor), '\n')

Tablica a:
 [[-0.55904888  1.20377133  0.14487737]
 [ 0.57694066  0.01327211  0.40258738]
 [-0.35591277  0.0770177   0.45265247]] 

Tablica b:
 [[-0.67230271  0.40107251 -0.0457552 ]
 [ 0.76225214  0.95731115 -0.34027763]
 [ 0.89671563  0.13214244 -0.06963209]] 

Mnożenie elementwise:
 [[ 0.37585007  0.48279959 -0.00662889]
 [ 0.43977425  0.01270554 -0.13699148]
 [-0.31915254  0.01017731 -0.03151914]] 

Mnożenie macierzy:
 [[ 1.42334115  0.94730904 -0.39412517]
 [-0.01675568  0.29729946 -0.05894724]
 [ 0.70388857 -0.00920232 -0.04144168]] 

Transpozycja:
 [[-0.55904888  0.57694066 -0.35591277]
 [ 1.20377133  0.01327211  0.0770177 ]
 [ 0.14487737  0.40258738  0.45265247]] 

Potęgowanie:
 [[3.12535648e-01 1.44906542e+00 2.09894516e-02]
 [3.32860522e-01 1.76148947e-04 1.62076595e-01]
 [1.26673899e-01 5.93172572e-03 2.04894256e-01]] 

Broadcasted multiply:
 [[-0.55904888  2.40754267  0.4346321 ]
 [ 0.57694066  0.02654422  1.20776213]
 [-0.35591277  0.15403539  1.3579574 ]] 

Broadcasted ad

Podstawowe operacje jak dodawanie, odejmowanie, mnożenie (element-wise), potęgowanie jest również dostępne korzystając z Python-owych operatorów (+, -, ...)

In [9]:
a = np.ones([3, 3])

print(a, '\n')
print((a * 2) ** 2, '\n')
print(a + 10, '\n')
print(a * 10, '\n')

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]] 

[[4. 4. 4.]
 [4. 4. 4.]
 [4. 4. 4.]] 

[[11. 11. 11.]
 [11. 11. 11.]
 [11. 11. 11.]] 

[[10. 10. 10.]
 [10. 10. 10.]
 [10. 10. 10.]] 



### Zadanie 0
Wykonaj poniższe polecenia:

In [10]:
arr = np.linspace(0, 20, 20)

#TODO: zmień kształt tablicy na (2, 10) - 2 wiersze po 10 wartości
arr = np.reshape(arr, [2,10])
print(arr, '\n')

#TODO: powiększ pierwszy wiersz o 2, drugi o 5. Operacja powinna być wykonana w 1 linii (zaalokuj zmienną a i wykorzystaj ją w operacji)
a = np.array([2, 5], dtype=np.float32).reshape(2,1)
arr = arr + a

print(arr, '\n')

#TODO: tablice NumPy można indeksować tak jak w innych językach programowania (np C, C++, Java)
#TODO: ''wybierz'' z tablicy drugi wiersz, podnieś do kwadratu jego elementy i wyświetl funkcją print()
arr[1] = arr[1]**2
print(arr)

[[ 0.          1.05263158  2.10526316  3.15789474  4.21052632  5.26315789
   6.31578947  7.36842105  8.42105263  9.47368421]
 [10.52631579 11.57894737 12.63157895 13.68421053 14.73684211 15.78947368
  16.84210526 17.89473684 18.94736842 20.        ]] 

[[ 2.          3.05263158  4.10526316  5.15789474  6.21052632  7.26315789
   8.31578947  9.36842105 10.42105263 11.47368421]
 [15.52631579 16.57894737 17.63157895 18.68421053 19.73684211 20.78947368
  21.84210526 22.89473684 23.94736842 25.        ]] 

[[  2.           3.05263158   4.10526316   5.15789474   6.21052632
    7.26315789   8.31578947   9.36842105  10.42105263  11.47368421]
 [241.06648199 274.86149584 310.87257618 349.09972299 389.54293629
  432.20221607 477.07756233 524.16897507 573.47645429 625.        ]]


### Zadanie 1
Napisz funkcję liniową dla wielu zmiennych (**X** ma rozmiar *n* - liczba zmiennych), jednego współczynnika kierunkowego **a** i jednego wyrazu wolnego **b**. Następnie sprawdź, czy funkcja działa również, gdy dla każdej zmiennej przypiszemy osobne wartości a i b.

In [11]:
x = np.array([1, 2, 3, 4, 5])
a = np.array(2)
b = np.array(-1)

def f1(x, a, b):
  return a*x + b

print(f1(x, a, b))

a = np.array([1, 5, 2, 3, 1])
b = np.array([5, 2, 3, 5, 1])

print(f1(x, a, b))

[1 3 5 7 9]
[ 6 12  9 17  6]


Mechanizm polegający na wykonaniu tej samej operacji dla wszystkich wartości w danej osi nazywa się **broadcasting**. Przykładowo, wynikiem mnożenia dwóch zmiennych o wymiarach [20] i [1] będzie zmienna o wymiarze [20], dla zmiennych o wymiarach [20, 1] i [1, 20] wynikiem będzie zmienna o wymiarach [20, 20] (a więc każdy element z pierwszej zmiennej, przemnożony zostanie przez wszystkie wartości (**osobno**) z drugiej zmiennej.

### Zadanie 2

Napisz funkcję, która będzie wykonywała kombinację liniową wektora zmiennych **X** oraz wektora **a**. Sprawdź, czy dla wielu wektorów **a** (kolumny tablicy **a**) funkcja będzie działała (jeśli nie, popraw funkcję).


In [12]:
x = np.array([2, 3, 5])
a = np.array([5, 1, 5])

def f2(x, a):
  return np.dot(x, a)

print(f2(x, a))

# for broadcast
x = np.expand_dims(x, 0)

a = np.array([
    [5, 2],
    [1, 2],
    [5, 1]
])

print(f2(x, a))

38
[[38 15]]


## Perceptron (Neuron)

Podstawowym elementem każdej sieci neuronowej jest perceptron. Jest to jednostka która wykonuje następującą operację:

$$z = \sum_i x_i * w_i + b$$
$$y = f(z)$$

Gdzie $x_i$ to i-te *wejście* perceptronu, $w_i$ to waga tego wejścia, natomiast $b$ to pewna stała. Wyjście funkcji liniowej $z$ jest następnie podawane na wejście pewnej funkcji aktywacji $f$.

Istnieje wiele funkcji aktywacji takich jak:
* skokowa (*funkcja progowa unipolarna*),
$${\displaystyle y(x)=\left\{{\begin{matrix}0 & dla & x\lt a\\1 & dla&x\geq a\\\end{matrix}}\right.}$$
* sigmoid,
$$y(x)={\frac {1}{1+e^{-\beta x}}}$$
* tangens hiperboliczny,
$$y(x)={\frac {2}{1+e^{-\beta x}}}-1={\frac {1-e^{-\beta x}}{1+e^{-\beta x}}}$$
* relu,
$$y(x)=max(0, x)$$


 Dla skokowej funkcji aktywacji zwyczajowo mówimy o perceptronie, dla innych funkcji aktywacji mówimy o **neuronie**.


---


Neuron wchodzi w skład wielu różnych elementów sieci neuronowej jak *konwolucja* czy *warstwa w pełni połączona* (fully connected).
W przypadku warstwy fully connected wzór rozszerza się trywialnie na:

$$z_j = \sum_i x_i * w_{ij} + b_j$$
$$y_j = f(z_j)$$

Gdzie $j$ oznacza j-ty neuron.
Jak widać powyżej, każdy z neuronów korzysta ze wszystkich wejść. Składnik $\sum_i x_i * w_{ij}$ to kombinacja liniowa (patrz zadanie 2).



---

### Zadanie 3
Zaimplementuj z wykorzystaniem biblioteki NumPy prosty perceptron (linear) oraz następujące funkcje aktywacji: **sigmoid**, **relu**.

In [13]:
def linear(x, w, b):
  return np.dot(x, w)+b


def sigmoid(z, beta=0.5):
  return 1/(1+np.exp(-beta*z))


def relu(z):
  return np.maximum(0, z)

# UWAGA: rozmiar wejściowy (1, 5) ma dodaną jedynkę tylko ze względów obliczeniowych
x = np.random.normal(0, 1, [1, 5])
w = np.random.normal(0, 1, [5, 2])
b = np.random.normal(0, 1, [2])

z = linear(x, w, b)

print('Linear activation:\n', z, '\n')
print('Sigmoid activation:\n', sigmoid(z), '\n')
print('Relu activation:\n', relu(z), '\n')

Linear activation:
 [[0.7340093  0.16301621]] 

Sigmoid activation:
 [[0.590735   0.52036575]] 

Relu activation:
 [[0.7340093  0.16301621]] 



## Uczenie neuronów

Podstawową metodą uczenia sieci neuronowych jest **metoda największego spadku**. Aby wykonać tę metodę niezbędne jest zdefiniowanie odpowiedniej **funkcji straty**. Funkcja straty określa jak bardzo nasza sieć myli się na danym przykładzie. Przykładowo jeśli neuron ma za zadanie odwzorować funkcję sinus to funkcją straty może być różnica pomiędzy estymowanym wyjściem neuronu ($y$) a pożądanym wyjściem ($y'$). Innym przykładem może byc klasyfikacja - sieć może estymować rozkład prawdopodobieństwa klas dla danego obrazka wejściowego (np. czy jest to kot czy pies), natomiast pożądanym wyjściem będzie **one-hot** wektor z zapalonym bitem na pozycji klasy prawdziwej. Wówczas jako funkcję straty możemy określić **cross-entropy**.

Dla różnego rodzaju sieci (i ich przeznczenia) można korzystać z różnych funkcji strat. Do najpopularniejszych należą:
* Mean Square Error:
$$L = \frac{1}{n}\sum_i^n(y - y')^2$$
gdzie $y$ to wyjście z neuronu (sieć), natomiast $y'$ to tzw. ground-truth - wartość, którą neuron (sieć) ma aproksymować,
* Mean Absolute Error:
$$L = \frac{1}{n}\sum_i^n |y - y'|$$
* Cross-Entropy:
$$L(p,q)=-\sum _{x}p(x)\,\log q(x)$$
gdzie p to pewien rozkład prawdopodobieństwa estymowany przez sieć, q to ground-truth rozkład prawdopodobieństwa. $x$ to przykład wejściowy.

### Zadanie 4
Zaimplementuj funkcję straty Mean Square Error (UWAGA: Mean Square Error jest liczony dla każdego przykładu (wiersza) osobno - przy obliczaniu średniej podaj odpowiedni parametr axis, wykorzystaj funkcję **np.mean**):

In [14]:
y = np.array([
    [5, 1, 2],
    [2, 2, 3]
])
y_hat = np.array([
    [1, 5, 2],
    [2, 3, 2]
])

def mse(y, y_hat):
  return np.mean((y-y_hat)**2, axis=1)

print('Mean Square Error:', mse(y, y_hat))

Mean Square Error: [10.66666667  0.66666667]


#### Aktualizacja wag

We wcześniejszych zadaniach można wyróżnić **zmienne uczące** oraz **propagowany sygnał**. Zmiennymi uczącymi nazywamy wszystkie parametry, które wchodzą w skład neuronów ale nie są podawane na wejściu. Z ich pomocą uczymy sieć aby produkowała pożądane wyjście na podstawie przykładów wejściowych. Przykłady wejściowe jak i rezultaty neuronów nazywamy propagowanym sygnałem. Jest to sygnał wykorzystany w procesie uczenia do określenia błędu jaki popełnił każdy z neuronów i w końcu do **obliczenia nowych wartości zmiennych**.

Wyliczenie nowych wartości zmiennych przebiega zgodnie z zasadą:

$$w_i' = w_i - \mu \frac{\partial L}{\partial w_i} $$

Gdzie $\mu$ to prędkość uczenia.

Jak widać największy problem stanowi obliczenie pochodnej funkcji straty po danej wadze. W przypadku jednowarstwowej sieci neuronowej otrzymujemy (korzystając z chain-rule):

$$\frac{\partial L}{\partial w_i} = \frac{\partial L}{\partial y} \frac{\partial y}{\partial w_i}$$

Wówczas ponownie korzystając z chain rule:

$$\frac{\partial L}{\partial w_i} = \frac{\partial L}{\partial y} \frac{\partial y}{\partial w_i} = \frac{\partial L}{\partial y} \frac{\partial y}{\partial z}\frac{\partial z}{\partial w_i}$$

Gdzie $\frac{\partial y}{\partial z}$ to pochodna funkcji aktywacji, natomiast $z = \sum_iw_ix_i + b$, dla której pochodna jest równa $x_i$, stąd:

$$\frac{\partial L}{\partial w_i} = \frac{\partial L}{\partial y} f'(z) x_i$$

Końcowo:

$$w_i' = w_i - \mu \frac{\partial L}{\partial y} f'(z) x_i $$

*Wskazówka do kolejnych zadań:*

* Zastępując $\frac{\partial L}{\partial y} f'(z) = \delta$ wzór można zapisać jako:
$$w_i' = w_i - \mu \delta x_i $$


---

### Zadanie 5
Zaimplementuj pochodne funkcji aktywacji: **relu** oraz **sigmoid** oraz funkcji liniowej. Dodatkowo zaimplementuj pochodną funkcji straty **Mean Square Error**.

**Podpowiedź:** tablice w NumPy można indeksować tablicą bool-ową. Przykład:


```
a = np.random.normal(0, 1, [3, 3])
a[a<0] = -1
a[a>=0] = 1
```

Powyższy przykład ustawia wartości w tablicy na -1, gdzie poprzednio była wartość ujemna, natomiast dla wartości dodatnich (lub 0) ustawia wartość 1.




In [15]:
def d_relu(z):
    return np.where(z >= 0, 1, 0)

def d_sigmoid(z, beta=0.5):
  help = np.exp(-beta*z)
  return (beta * help)/((1+help)**2)

# pochodna po zmiennych tylko W
def d_linear(x, w, b):
  return x

def d_mse(y, y_hat):
    return -2 * np.mean(y - y_hat, axis=1)


### Zadanie 6
Zaimplementuj funkcję obliczającą nową wartość dla zmiennych **w**.

In [16]:
mu = 1e-2

# UWAGA: rozmiar wejściowy (1, 5) ma dodaną jedynkę tylko ze względów obliczeniowych
x = np.random.normal(0, 1, [1, 5])
w = np.random.normal(0, 1, [5, 1])
b = np.random.normal(0, 1, [1])

y_hat = np.ones([1, 1], dtype=np.float32)

z = linear(x, w, b)
y = relu(z)

print('Z:\n', z, '\n')
print('Y:\n', y)

delta = d_relu(z) * d_mse(y, y_hat)

print('Delta:\n', delta, '\n')


def update(variable, mu, delta, x):
  return variable - mu*delta*x.T

w_new = update(w, mu, delta, x)

print('Wagi przed:\n', w, '\n')
print('Wagi po:\n', w_new)

Z:
 [[-1.84028176]] 

Y:
 [[0.]]
Delta:
 [[0.]] 

Wagi przed:
 [[-0.59934506]
 [ 0.14634561]
 [-0.31878978]
 [-0.18668508]
 [-0.93784404]] 

Wagi po:
 [[-0.59934506]
 [ 0.14634561]
 [-0.31878978]
 [-0.18668508]
 [-0.93784404]]


### Zadanie 7*
Wyznacz regułę aktualizacji dla zmiennych **b**.

In [19]:
def update_b(variable, mu, delta):
  return variable - mu*delta

b_new = update_b(b, mu, delta)

print('Wagi przed:\n', b, '\n')
print('Wagi po:\n', b_new)

Wagi przed:
 [-0.73166449] 

Wagi po:
 [[-0.73166449]]
